### Importing Libraries & Models

In [ ]:
import os
import sys
import pprint
import tarfile
import zipfile
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import six.moves.urllib as urllib
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.python.util import compat
from tensorflow.core.protobuf import saved_model_pb2
from google.protobuf import text_format
from collections import defaultdict
from io import StringIO
from IPython.display import display

# getting the tensorflow models
! rm -rf ./models && git clone https://github.com/tensorflow/models.git \
    && cd models/research \
    && protoc object_detection/protos/*.proto --python_out=. \
    && cp object_detection/packages/tf2/setup.py . && \
    python3 -m pip install --use-feature=2020-resolver .

from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import dataset_util, label_map_util
from object_detection.protos import string_int_label_map_pb2

# upgrading tensorflow gpu and the datasets
!pip install --upgrade tensorflow-gpu==2.2.0 --user
!pip uninstall tensorflow-datasets
!pip install tensorflow-datasets==4.0.0

Cloning into 'models'...
remote: Enumerating objects: 64364, done.
remote: Total 64364 (delta 0), reused 0 (delta 0), pack-reused 64364
Receiving objects: 100% (64364/64364), 575.02 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (45024/45024), done.
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 11.1 MB/s 
     |████████████████████████████████| 352 kB 63.6 MB/s 
     |████████████████████████████████| 1.8 MB 49.8 MB/s 
     |████████████████████████████████| 1.2 MB 67.8 MB/s 
     |████████████████████████████████| 1.1 MB 64.8 MB/s 
     |█████████████████████

Found existing installation: tensorflow-datasets 4.0.1


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
pip install tensorflow-object-detection-api

### Loading the Model & Getting our Test Images

In [ ]:
# function to load a particular model
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.compat.v2.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [ ]:
# list of the strings that is used to add correct label for each box
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# getting the paths of all the images
from fastai.imports import *
PATH_TO_TEST_IMAGES_DIR = Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
# loading the presaved mobilenent v1 model
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

In [ ]:
# printing out the type of the results
print(detection_model.inputs)
detection_model.output_dtypes

### Printing out the Results

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]
  output_dict = model(input_tensor)
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path,class_id):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  for i,x in enumerate(output_dict['detection_classes']):
    if x==class_id and output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  
  display(Image.fromarray(image_np))

In [ ]:
# testing it on images
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path, 1)

### Testing it on a new Image
#### *You can take any jpg image.*

In [ ]:
show_inference(detection_model, '/content/test_img.jpeg', 1)